In [1]:
%reload_ext autoreload
%autoreload 2

## Setup

In [2]:
from math import isclose
from ipygpulogger import IPyGPULogger
import ipygpulogger
import re, numpy as np

In [3]:
def consume_cpu(n): return np.ones((n, n))

In [4]:
# --------------------------------------------------------------------- #
# the following functions work with the captured output
# output is captured by `%%capture output` from a cell before
        
def get_used_size(output):
    match = re.findall(r'Gen: +([\d\.]+) +([\d\.]+) +([\d\.]+) MB', output)
    (consumed, peaked, total) = map(float, match[0])
    return consumed, peaked, total

def check_report(output, consumed_real, peaked_real):
    # basic checks
    to_match = [r'Consumed', 'Peaked']
    for s in to_match: assert re.search(s, output), f"expecting string: {s}"
        
    consumed, peaked, total = get_used_size(output)
    
    # compare: numbers are within 2% equal
    assert isclose(consumed, consumed_real, rel_tol=0.03), f"Consumed RAM {consumed} == {consumed_real}"


In [5]:
"""test_version"""
assert ipygpulogger.__version__, "version check"

'test_version'

## Report and Data tests

In [6]:
il = IPyGPULogger().start()

RAM: Consumed Peaked  Used Total in 0.001s (In [6])
Gen:        2      0      172 MB
GPU:        0      0     1962 MB


In [7]:
%%capture output
x1 = consume_cpu(2**12)

RAM: Consumed Peaked  Used Total in 0.053s (In [7])
Gen:        0      0      300 MB
GPU:        0      0     1962 MB


In [8]:
"""test_report"""
output = str(output)
#print(output)
check_report(output, 127, 0)

'test_report'

RAM: Consumed Peaked  Used Total in 0.001s (In [8])
Gen:        0      0      300 MB
GPU:        0      0     1962 MB


In [9]:
"""test_data"""
consumed, peaked, total = get_used_size(output)

gen_mem_used_delta, gen_mem_peaked, gen_mem_used, gpu_mem_used_delta, gpu_mem_peaked, gpu_mem_used, time_delta = il.data

assert isclose(total, gen_mem_used, rel_tol=0.01), f"Total used RAM {total} == {gen_mem_used}"

'test_data'

RAM: Consumed Peaked  Used Total in 0.031s (In [9])
Gen:        0      0      300 MB
GPU:        0      0     1962 MB


In [10]:
il.stop()

In [11]:
%%javascript # prevent committing an unsaved notebook
IPython.notebook.save_notebook()

<IPython.core.display.Javascript object>